```
**Medical Image Classification using Pretrained ResNet50**
```

## 1. Import Libraries

In [1]:
import os
import numpy as np
import glob
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import GlobalMaxPooling2D, Dense, Input
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

## 2. Load Image Paths

In [2]:
# Define the base folder
base_folder = 'Multimodal_images'
jpg_files = glob.glob(os.path.join(base_folder, '**', '*.jpg'), recursive=True)
print(jpg_files)

['Multimodal_images\\cyanosis\\Image_1.jpg', 'Multimodal_images\\cyanosis\\Image_10.jpg', 'Multimodal_images\\cyanosis\\Image_11.jpg', 'Multimodal_images\\cyanosis\\Image_13.jpg', 'Multimodal_images\\cyanosis\\Image_14.jpg', 'Multimodal_images\\cyanosis\\Image_18.jpg', 'Multimodal_images\\cyanosis\\Image_19.jpg', 'Multimodal_images\\cyanosis\\Image_2.jpg', 'Multimodal_images\\cyanosis\\Image_20.jpg', 'Multimodal_images\\cyanosis\\Image_29.jpg', 'Multimodal_images\\cyanosis\\Image_3.jpg', 'Multimodal_images\\cyanosis\\Image_31.jpg', 'Multimodal_images\\cyanosis\\Image_32.jpg', 'Multimodal_images\\cyanosis\\Image_37.jpg', 'Multimodal_images\\cyanosis\\Image_4.jpg', 'Multimodal_images\\cyanosis\\Image_43.jpg', 'Multimodal_images\\cyanosis\\Image_5.jpg', 'Multimodal_images\\cyanosis\\Image_6.jpg', 'Multimodal_images\\dry scalp\\Image_13.jpg', 'Multimodal_images\\dry scalp\\Image_18.jpg', 'Multimodal_images\\dry scalp\\Image_19.jpg', 'Multimodal_images\\dry scalp\\Image_22.jpg', 'Multimodal

In [3]:
len(jpg_files)

532

## 3. Load Images & Labels for Given Paths

In [4]:
# Function to load and preprocess images
def load_and_preprocess_images(image_paths, target_size=(224, 224)):
    images = []
    labels = []
    for path in image_paths:
        image = load_img(path, target_size=target_size)   #module come from tensorflow.keras.preprocessing.image
        image = img_to_array(image)  # (height x width x channels) = (224,224,3)
        images.append(image)
        label = os.path.basename(os.path.dirname(path))  # Assuming folder name is the label
        labels.append(label)
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

## 4. Function Call for Load Image and Labels(Encode Labels)

In [5]:
# Load the images and their labels
image_paths =jpg_files
images, labels = load_and_preprocess_images(image_paths)
print("Shape of images:", images.shape)
print("list of Image:", images)
print("list of labels paths:", labels)

Shape of images: (532, 224, 224, 3)
list of Image: [[[[  9.  13.  12.]
   [  6.  10.   9.]
   [  7.  11.  10.]
   ...
   [177. 226. 205.]
   [182. 227. 207.]
   [181. 230. 209.]]

  [[  6.  10.   9.]
   [  4.   8.   7.]
   [  6.  10.   9.]
   ...
   [177. 226. 205.]
   [182. 227. 207.]
   [181. 230. 209.]]

  [[  7.  11.  10.]
   [  5.   9.   8.]
   [  5.   9.   8.]
   ...
   [179. 228. 207.]
   [180. 225. 205.]
   [179. 228. 207.]]

  ...

  [[103. 168. 206.]
   [ 92. 156. 192.]
   [ 91. 155. 193.]
   ...
   [206. 171. 149.]
   [201. 174. 147.]
   [200. 173. 144.]]

  [[ 66. 135. 168.]
   [ 55. 121. 155.]
   [ 33.  93. 129.]
   ...
   [203. 171. 148.]
   [206. 176. 152.]
   [201. 175. 150.]]

  [[ 59. 125. 159.]
   [ 48. 111. 146.]
   [ 41.  99. 136.]
   ...
   [203. 171. 148.]
   [211. 181. 157.]
   [201. 175. 150.]]]


 [[[129. 144. 111.]
   [130. 145. 112.]
   [120. 137. 105.]
   ...
   [146. 152. 124.]
   [163. 173. 149.]
   [168. 183. 152.]]

  [[148. 163. 132.]
   [138. 153. 120

In [6]:
print(len(images))
print(len(images[0]))
print(len(images[0][0]))
print(len(images[0][0][0]))

532
224
224
3


## 5. Preprocess images for ResNet50

In [7]:
# Preprocess images for ResNet50
images = preprocess_input(images) 
#For ResNet50, this involves converting the pixel values from a range of [0, 255] to a range suitable for the model.
# "(1)Convert RGB to BGR and (2)convert center at 0(i,e: Scale value around 0) "

In [8]:
images[0]

array([[[ -91.939    , -103.779    , -114.68     ],
        [ -94.939    , -106.779    , -117.68     ],
        [ -93.939    , -105.779    , -116.68     ],
        ...,
        [ 101.061    ,  109.221    ,   53.32     ],
        [ 103.061    ,  110.221    ,   58.32     ],
        [ 105.061    ,  113.221    ,   57.32     ]],

       [[ -94.939    , -106.779    , -117.68     ],
        [ -96.939    , -108.779    , -119.68     ],
        [ -94.939    , -106.779    , -117.68     ],
        ...,
        [ 101.061    ,  109.221    ,   53.32     ],
        [ 103.061    ,  110.221    ,   58.32     ],
        [ 105.061    ,  113.221    ,   57.32     ]],

       [[ -93.939    , -105.779    , -116.68     ],
        [ -95.939    , -107.779    , -118.68     ],
        [ -95.939    , -107.779    , -118.68     ],
        ...,
        [ 103.061    ,  111.221    ,   55.32     ],
        [ 101.061    ,  108.221    ,   56.32     ],
        [ 103.061    ,  111.221    ,   55.32     ]],

       ...,

      

## 6. Encode Labels

In [9]:
se=set()
for i in labels:
    se.add(i)
print("Unique label:", len(se))
print(se)

Unique label: 18
{np.str_('itichy eyelid'), np.str_('hand lump'), np.str_('swollen eye'), np.str_('swollen tonsils'), np.str_('cyanosis'), np.str_('foot swelling'), np.str_('mouth ulcers'), np.str_('skin dryness'), np.str_('skin growth'), np.str_('eye_inflamation'), np.str_('lip swelling'), np.str_('skin irritation'), np.str_('skin rash'), np.str_('edema'), np.str_('neck swelling'), np.str_('knee swelling'), np.str_('dry scalp'), np.str_('eye_redness')}


In [10]:
# Encode labels
label_encoder = LabelEncoder()
print(label_encoder)
#For example, if labels are ['cat', 'dog', 'mouse'], it might convert them to [0, 1, 2]
labels_encoded = label_encoder.fit_transform(labels) 
print(labels_encoded)
# len(labels_encoded)
#Convert Numerical Labels to One-Hot Encoding:
labels_categorical = to_categorical(labels_encoded)
labels_categorical

LabelEncoder()
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3
  3  3  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  5
  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5
  5  5  5  5  5  5  5  5  5  5  5  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7
  7  7  7  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
 11 11 11 11 11 11 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12
 12 12 12 12 12 13 13 13 13 13 13 13

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], shape=(532, 18))

In [11]:
print(len(labels_categorical))
print(len(labels_categorical[0]))

532
18


## 7. Split Train/Test

In [12]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels_categorical, test_size=0.2, random_state=42)

# A. Apply ResNet50 Model

## 8. Build ResNet50 Model

In [13]:
# Load ResNet50 model without the top fully connected layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
# Freeze the base model layers to avoid retraining them. 
# Means weight unchange(take weight from orginal renest50) in new sequential model

# base_model's weights will NOT be updated during training. Only the layers you add on top will be trained

for layer in base_model.layers:
    layer.trainable = False  # <--- IMPORTANT For Transfer Learning

## If you do NOT do this you do not get good accuracy. Beacusr the ResNet50 train on many many Images

*If you do NOT do this you do not get good accuracya. Because the ResNet50 train on many many Images*

In [37]:
print("ResNet50 Base Model Input:",base_model.input)
print("ResNet50 Base Model Output:",base_model.output)

ResNet50 Base Model Input: <KerasTensor shape=(None, 224, 224, 3), dtype=float32, sparse=False, ragged=False, name=keras_tensor>
ResNet50 Base Model Output: <KerasTensor shape=(None, 7, 7, 2048), dtype=float32, sparse=False, ragged=False, name=keras_tensor_174>


In [ ]:
# Add custom layers on top of the base model
model = Sequential([
    base_model,                     # input=(224,224,3), output=(7,7,2048)
    GlobalMaxPooling2D(),           # input=(7,7,2048), output=(,2048)
    Dense(1024, activation='relu'), # input=(,2048), output=(,1024)
    Dense(len(labels_categorical[0]), activation='softmax') # input=(,1024), output=(,18)
])

## 9. Compile Model

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

## 10. Train Model

In [17]:
# Train the model
model.fit(
    X_train, y_train, 
    epochs=10, validation_data=(X_test, y_test))


Epoch 1/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 54s 3s/step - accuracy: 0.2494 - loss: 23.1044 - val_accuracy: 0.2523 - val_loss: 14.5475
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 37s 3s/step - accuracy: 0.6000 - loss: 4.7192 - val_accuracy: 0.5047 - val_loss: 3.8587
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 35s 3s/step - accuracy: 0.7529 - loss: 1.1493 - val_accuracy: 0.5981 - val_loss: 2.8078
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 38s 3s/step - accuracy: 0.8659 - loss: 0.5309 - val_accuracy: 0.6355 - val_loss: 1.8322
Epoch 5/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.9341 - loss: 0.2107 - val_accuracy: 0.5794 - val_loss: 2.0540
Epoch 6/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.9576 - loss: 0.1392 - val_accuracy: 0.5327 - val_loss: 1.9085
Epoch 7/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.9835 - loss: 0.0604 - val_accuracy: 0.6075 - val_loss: 1.8638
Epoch 8/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.9788 - loss: 0.0471 - val_accuracy: 0.5421 - val_los

## 11. Evaluate Model

In [18]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy * 100:.2f}%")

4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.6075 - loss: 1.8946
Test accuracy: 60.75%


In [19]:
# Save the model
# model.save('custom_resnet50_model.h5')

**Predict Model in New Data**

In [20]:
image, labels = load_and_preprocess_images(['red-irritated-eye-showing-prominent-260nw-2597914653.webp'])
preds = model.predict(image)
pred_class = np.argmax(preds)
pred_label = label_encoder.inverse_transform([pred_class])[0]
print(pred_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
eye_redness


# B. Apply VGG16 model

In [21]:
from tensorflow.keras.applications.vgg16 import VGG16

## 12. Build ResNet50 Model

In [22]:
# Load VGG16 without top layers
base_model_vgg16 = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

In [35]:
print("VGG16 Base Model Input:",base_model_vgg16.input)
print("VGG16 Base Model Output:",base_model_vgg16.output)

VGG16 Base Model Input: <KerasTensor shape=(None, 224, 224, 3), dtype=float32, sparse=False, ragged=False, name=keras_tensor_180>
VGG16 Base Model Output: <KerasTensor shape=(None, 7, 7, 512), dtype=float32, sparse=False, ragged=False, name=keras_tensor_198>


In [23]:
base_model_vgg16.layers

[<InputLayer name=input_layer_2, built=True>,
 <Conv2D name=block1_conv1, built=True>,
 <Conv2D name=block1_conv2, built=True>,
 <MaxPooling2D name=block1_pool, built=True>,
 <Conv2D name=block2_conv1, built=True>,
 <Conv2D name=block2_conv2, built=True>,
 <MaxPooling2D name=block2_pool, built=True>,
 <Conv2D name=block3_conv1, built=True>,
 <Conv2D name=block3_conv2, built=True>,
 <Conv2D name=block3_conv3, built=True>,
 <MaxPooling2D name=block3_pool, built=True>,
 <Conv2D name=block4_conv1, built=True>,
 <Conv2D name=block4_conv2, built=True>,
 <Conv2D name=block4_conv3, built=True>,
 <MaxPooling2D name=block4_pool, built=True>,
 <Conv2D name=block5_conv1, built=True>,
 <Conv2D name=block5_conv2, built=True>,
 <Conv2D name=block5_conv3, built=True>,
 <MaxPooling2D name=block5_pool, built=True>]

In [24]:
# Freeze the base model layers to avoid retraining them. 
# Means weight unchange(take weight from orginal renest50) in new sequential model

# base_model's weights will NOT be updated during training. Only the layers you add on top will be trained

for layer in base_model_vgg16.layers:
    layer.trainable = False  # <--- IMPORTANT For Transfer Learning


In [25]:
# Sequential model structure (as requested)
model_vgg16 = Sequential([
    base_model_vgg16,                    # Input: (224, 224, 3), output: (7,7,512)
    GlobalMaxPooling2D(),                # Input: (7, 7, 512), output: (,512)
    Dense(1024, activation='relu'),      # Input: (,512), output: (,1024)
    Dense(len(labels_categorical[0]), activation='softmax')  # Input: (,1024), output: (,18)
])

# VGG16 performs multiple convolution + max-pool operations.
# After all pooling, spatial size gets reduced from 224 ⟶ 112 ⟶ 56 ⟶ 28 ⟶ 14 ⟶ 7

## 13. Compile Model

In [ ]:
model_vgg16.compile(
    optimizer="adam", 
    loss="categorical_crossentropy", 
    metrics=["accuracy"]
)
print(model_vgg16.summary())

## 14. Train Model

In [27]:
# Train the model_vgg16
model_vgg16.fit(
    X_train, y_train, 
    epochs=10, validation_data=(X_test, y_test))


Epoch 1/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 60s 4s/step - accuracy: 0.2235 - loss: 32.5000 - val_accuracy: 0.4860 - val_loss: 7.5393
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 52s 4s/step - accuracy: 0.6847 - loss: 4.6094 - val_accuracy: 0.5327 - val_loss: 8.6975
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 59s 4s/step - accuracy: 0.8141 - loss: 1.8329 - val_accuracy: 0.4953 - val_loss: 10.1151
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 54s 4s/step - accuracy: 0.8635 - loss: 1.1833 - val_accuracy: 0.5701 - val_loss: 6.0915
Epoch 5/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 56s 4s/step - accuracy: 0.9035 - loss: 0.8757 - val_accuracy: 0.6355 - val_loss: 6.7679
Epoch 6/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 57s 4s/step - accuracy: 0.9412 - loss: 0.4491 - val_accuracy: 0.5701 - val_loss: 7.3859
Epoch 7/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 56s 4s/step - accuracy: 0.9694 - loss: 0.1498 - val_accuracy: 0.5888 - val_loss: 7.4665
Epoch 8/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 54s 4s/step - accuracy: 0.9859 - loss: 0.0747 - val_accuracy: 0.5981 - val_los

## 15. Evaluate vgg16 model

In [28]:
# Evaluate the model_vgg16
loss, accuracy = model_vgg16.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy * 100:.2f}%")

4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.6168 - loss: 7.3759
Test accuracy: 61.68%


In [29]:
# Save the model
# model_vgg16.save('custom_vgg16_model.h5')

**Predict VGG16 Model in New Data**

In [30]:
image, labels = load_and_preprocess_images(['red-irritated-eye-showing-prominent-260nw-2597914653.webp'])

preds_vgg16 = model_vgg16.predict(image)
    
# Get the predicted class label
pred_class_vgg16 = np.argmax(preds_vgg16)
    
# Map predicted class index to class label
pred_label_vgg16 = label_encoder.inverse_transform([pred_class_vgg16])[0]

print(pred_label_vgg16)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
eye_inflamation
